In [ ]:
# Code taken from https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/

In [1]:
#Loading NLTK
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

import pandas as pd


In [ ]:
#pip install -U textblob
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install numpy

In [35]:
#Read in dictionary file
Dictpath = '~torisugden/Desktop/MA5854 - Project 2/ClinicalAbbrs.xlsx' 
Dictdata = pd.read_excel (Dictpath)
dictdf = pd.DataFrame(Dictdata)
dictdf.head(10)


,Abbreviation,Meaning
0,Ab,Antibody
1,AB,Antiobiotic
2,Abdo,Abdomen
3,abdo,abdomen
4,ABG,Arterial blood gases
5,mum,mother
6,wks,weeks
7,Wks,weeks
8,yo,years old
9,pt,patient


In [36]:
#Set up dictionary
my_dict = dict(zip(dictdf['Abbreviation'], dictdf['Meaning']))
my_dict = {r"\b{}\b".format(k): v for k, v in my_dict.items()}
firstnpairs = {k: my_dict[k] for k in list(my_dict)[:10]}
print(firstnpairs)

{'\\bAb\\b': 'Antibody', '\\bAB\\b': 'Antiobiotic', '\\bAbdo\\b': 'Abdomen', '\\babdo\\b': 'abdomen', '\\bABG\\b': 'Arterial blood gases', '\\bmum\\b': 'mother', '\\bwks\\b': 'weeks', '\\bWks\\b': 'weeks', '\\byo\\b': 'years old', '\\bpt\\b': 'patient'}


In [37]:
#Read in file

#Filepath = 'https://jcudata.s3-ap-southeast-2.amazonaws.com/ED_Patients_Data.xlsx'
Filepath = '~torisugden/Desktop/MA5854 - Project 2/ED_Patients_Data_small.xlsx' 

#LookAt = 'PRESENTING_PROBLEM'
LookAt = 'TRIAGE_PRESENTING_INFORMATION'
#LookAt = 'TRIAGE_PRESENTING_ADDITIONAL_INFORMATION'

data = pd.read_excel (Filepath)
df = pd.DataFrame(data, columns= ['PATIENT_ID','TRIAGE_DT_TM','PRESENTING_PROBLEM', 'TRIAGE_PRESENTING_INFORMATION', 'TRIAGE_PRESENTING_ADDITIONAL_INFORMATION'])
#print (df)


ShowNum = 5

# Drop observations where LookAt is blank.
df = df[df[LookAt].notnull()]

In [38]:
df.head(ShowNum)

,PATIENT_ID,TRIAGE_DT_TM,PRESENTING_PROBLEM,TRIAGE_PRESENTING_INFORMATION,TRIAGE_PRESENTING_ADDITIONAL_INFORMATION
0,1dee72182d87b34cb330bf12d065cd021573ef09,2016-12-31T13:12:00Z,Pregnancy related problem,5 wks pregnant G2P1 presented with lower abdo ...,pmx ; nil
1,1c3bbdcef1bd6ef3fdfd2aba0ab848d970cc22fa,2016-12-31T13:19:00Z,"Pain, abdominal",Pt presented with sudden onset of generalised ...,"pt alert, well perfused, looks anxious. pmx ;..."
2,081e68d70b005d89fbce3690f8a3de87e4c3755b,2016-12-31T13:52:00Z,"Pain, limb lower / hip",pt presented with R ankle swelling since yeste...,"pt alert, well perfused, nil analgesia taken. ..."
3,46d73dbb85071edf3301cfb5fb0fe855592147eb,2016-12-31T14:08:00Z,Unwell,pt BIBA from church with sudden onset of dizz...,pmx ; nil
4,c8d82af5233c74cf91867f9ee9817dfa69304f79,2016-12-31T14:28:00Z,"Pain, abdominal",BIB mum with sudden onset of generalised abdo...,"child alert,well perfused, afebrile. wa sgiven..."


In [39]:
#Number of words in each text string
df['word_count'] = df[LookAt].apply(lambda x: len(str(x).split(" ")))
df[[LookAt,'word_count']].head(ShowNum)

,TRIAGE_PRESENTING_INFORMATION,word_count
0,5 wks pregnant G2P1 presented with lower abdo ...,32
1,Pt presented with sudden onset of generalised ...,38
2,pt presented with R ankle swelling since yeste...,37
3,pt BIBA from church with sudden onset of dizz...,43
4,BIB mum with sudden onset of generalised abdo...,25


In [40]:
#Number of stopwords in each text string
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['stopwords'] = df[LookAt].apply(lambda x: len([x for x in x.split() if x in stop]))
df[[LookAt,'stopwords']].head(ShowNum)


,TRIAGE_PRESENTING_INFORMATION,stopwords
0,5 wks pregnant G2P1 presented with lower abdo ...,3
1,Pt presented with sudden onset of generalised ...,8
2,pt presented with R ankle swelling since yeste...,10
3,pt BIBA from church with sudden onset of dizz...,10
4,BIB mum with sudden onset of generalised abdo...,5


In [41]:
#Run words through dictionary to expand abbreviations

df[LookAt] = df[LookAt].replace(my_dict, regex=True)
df[LookAt].head(ShowNum)


0    5 weeks pregnant G2P1 presented with lower abd...
1    patient presented with sudden onset of general...
2    patient presented with right ankle swelling si...
3    patient Brought in by Ambulance from church wi...
4    Brought in by  mother with  sudden onset of ge...
Name: TRIAGE_PRESENTING_INFORMATION, dtype: object

In [42]:
#Lower Case
df[LookAt] = df[LookAt].apply(lambda x: " ".join(x.lower() for x in x.split()))
df[LookAt].head(ShowNum)

0    5 weeks pregnant g2p1 presented with lower abd...
1    patient presented with sudden onset of general...
2    patient presented with right ankle swelling si...
3    patient brought in by ambulance from church wi...
4    brought in by mother with sudden onset of gene...
Name: TRIAGE_PRESENTING_INFORMATION, dtype: object

In [43]:
#Remove punctuation
df[LookAt] = df[LookAt].str.replace('[^\w\s]','')
df[LookAt].head(ShowNum)


0    5 weeks pregnant g2p1 presented with lower abd...
1    patient presented with sudden onset of general...
2    patient presented with right ankle swelling si...
3    patient brought in by ambulance from church wi...
4    brought in by mother with sudden onset of gene...
Name: TRIAGE_PRESENTING_INFORMATION, dtype: object

In [44]:
#Remove Stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
df[LookAt] = df[LookAt].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df[LookAt].head(ShowNum)

0    5 weeks pregnant g2p1 presented lower abdomen ...
1    patient presented sudden onset generalised abd...
2    patient presented right ankle swelling since y...
3    patient brought ambulance church sudden onset ...
4    brought mother sudden onset generalised abdome...
Name: TRIAGE_PRESENTING_INFORMATION, dtype: object

In [55]:
#Spellcheck   NOTE THIS IS SET TO ONLY DO THE FIRST 100 AT THE MOMENT!!!!!!!
from textblob import TextBlob
df[LookAt][:100].apply(lambda x: str(TextBlob(x).correct()))

0     5 weeks pregnant gap presented lower abdomen p...
1     patient presented sudden onset generalised abd...
2     patient presented right ankle swelling since y...
3     patient brought ambulance church sudden onset ...
4     brought mother sudden onset generalised abdome...
                            ...                        
95                                             cat bite
96                                               unwell
97                          fever decreased oral intake
98                            abdomen pain quadriplegia
99                             left upper quarante pain
Name: TRIAGE_PRESENTING_INFORMATION, Length: 100, dtype: object

In [56]:
#Word Frequency
freq = pd.Series(' '.join(df[LookAt]).split()).value_counts()[:10]
freq

patient     558
pain        519
nil         367
presents    270
since       246
left        231
brought     231
right       204
last        191
co          150
dtype: int64